# This notebook has codes implemented for getting ground truth data from ten manually annotated seed images.

All the files required for this notebook are in the '**ManualAnnotation**' directory located in the main directory. ManualAnnotation has three sub-directories:

- **Images**: This subdirectory contains 0ten preprocessed images with manual annotation.
- **JsonFiles**: This subdirectory contains two JSON files:
    -  metadata_1.json has an annotation for five images and metadata_2.json has an annotation for the other five images.
- **GroundTruth**: This is the directory where the data frame containing ground truth for each seed from manually annotated images will be saved after running the following notebook. The final frame will be saved in the following format: <imagename_ground_truth.csv>. Each ground truth file will contain 11 columns.
    - ROI: The arbitrary serial number assigned to each seed in an image.
    - Area: The number of pixels that belong to each seed.
    - Major_axis_length: The longest distance between two points on the major axis of the seed mask.
    - Minor_axis_length: The longest distance between two points on the minor axis of the seed mask.
    - y1,x1,y2,x2: bounding box pixel coordinates surround each seed.
    - Blue: Average blue intensity across pixels belonging to each seed.
    - Green: Average green intensity across pixels belonging to each seed.
    - Red: Average red intensity across pixels belonging to each seed.

In [15]:
import warnings
warnings.filterwarnings('ignore')  #general warning suppression
import random
import sys
import numpy as np
import json
import cv2 as cv
import os

from skimage.io import imread
from skimage.draw import polygon
import pandas as pd
from skimage.measure import label, regionprops, regionprops_table
from skimage import data, filters, measure, morphology, io, color

import tensorflow as tf ##we need tensorflow 2.10.0
print(tf.__version__)
import keras.backend as K ##we need keras 2.1.0

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  #tensorflow deprecration warning suppression
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
# import pytesseract
from PIL import ImageTk,Image, ImageDraw, ImageFilter

In [12]:
if not os.path.exists("../Mask-RCNN"):
    !git clone https://github.com/matterport/Mask_RCNN.git
    print("Cloned repository")
else:
    print("Mask_RCNN directory already exists")

Mask_RCNN directory already exists


In [13]:
sys.path.append("../Mask-RCNN") 

from mrcnn.visualize import display_images
from mrcnn.model import log
import mrcnn.model as modellib
from mrcnn.config import Config
from mrcnn import utils, visualize
print("done")

done


In [14]:
real_world_dir = "../ManualAnnotation/Images/"
json_files = "../ManualAnnotation/JsonFiles/"
Dataframe="../ManualAnnotation/Groundtruth/"

json_dir=os.path.join(json_files,random.choice(os.listdir(json_files))) ##Randomly choose one .json file out of two files.

In [20]:
with open(json_dir) as f:
    jsondata = json.load(f)

files = [x["file_name"] for x in jsondata["images"]]
print(files)

['4305.jpg', '4567.jpg', '4762.jpg', '4898.jpg', '5021.jpg']


In [23]:
for imagenum in files:
    print(imagenum)
    image = cv.imread(real_world_dir+str(imagenum))
    image = cv.cvtColor(img_bgr, cv.COLOR_BGR2RGB)
    myid=[]
    instance_masks = []
    class_ids = []
    bbox=[]
    for i in jsondata['images']:
        if i["file_name"]==str(imagenum):
            for j in jsondata['annotations']:
                if i["id"]==j["image_id"]:
                    x1=j["bbox"][0]
                    y1=j["bbox"][1]
                    x2=x1+j["bbox"][2]
                    y2=y1+j["bbox"][3]
                    list1=[y1,x1,y2,x2]
                    bbox.append(list1)
                    class_ids.append(j["category_id"])
                    myid.append(j["segmentation"])
            for segmentation in myid:
                mask = np.zeros(image.shape[:2])
                mask=Image.fromarray(mask)
                mask_draw = ImageDraw.Draw(mask)
                mask_draw.polygon(segmentation[0], outline=1, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
    class_ids=np.array(class_ids,dtype=np.int32)
    bbox=np.array(bbox,dtype=np.int32)
    instance_masks=np.array(instance_masks, dtype=np.int8)
    mask=np.transpose(instance_masks, (1,2,0))
    # r=model.detect([image], verbose=0)[0]
    count=mask.shape[-1]
    
    Morphological_ground=[]
    for i in range(bbox.shape[0]):
        singlemask=mask[:,:,i]
        b=bbox[i]
        props = regionprops(singlemask.astype(int))[0]
        area = props.area
        major_axis_length = props.major_axis_length
        minor_axis_length = props.minor_axis_length
        singlemask=singlemask.astype(bool)

        color=pd.DataFrame(image[singlemask])
        B = (color.mean()[2])
        G = (color.mean()[1])
        R = (color.mean()[0])
        
        
        Morphological_ground.append(
            {
                'ROI':i+1,
                'Area':area,
                'Major_axis_length':major_axis_length,
                'Minor_axis_length':minor_axis_length,
                'y1':b[0],
                'x1':b[1],
                'y2':b[2],
                'x2':b[3],
                'Blue':B,
                'Green':G,
                'Red':R
            }
        )

    print(f'Image Name: {imagenum}')
    
    visualize.display_instances(
        image, bbox, mask, class_ids,
        ["",""], ["" for x in range(len(bbox))],
        show_bbox=True, show_mask=True,
        title="")
    
    ground=pd.DataFrame(Morphological_ground)
    ground.to_csv(Dataframe+str(imagenum)+"_ground_truth"+".csv", index=False)
    
print("doneall")